# HW1
## Question 1

```
python ANN.py   --h1    <# of neurons in the first hidden layer>
                --h2    <# of neurons in the second hidden layer>             
                --lr    <learning rate>
                --epoch <epoch size>
```

In [3]:
from ANN import *
# read the data
X_train, y_train = read_dataset('./data/MNIST_X_train.csv', './data/MNIST_y_train.csv')
X_test, y_test = read_dataset('./data/MNIST_X_test.csv', './data/MNIST_y_test.csv')
X_train_norm, X_test_norm = normalize_features(X_train, X_test)
y_train_ohe, y_test_ohe = one_hot_encoder(y_train, y_test)
# tune the hyper parameters
for h1 in [50,100,200]:
    for h2 in [50,100,200]:
        for lr in [0.0005,0.001,0.005]:
            for epoch in [50,100,200,300]:
                myANN = twolayer_NN(X_train_norm, y_train_ohe, h1=h1, h2=h2, lr=lr)
                epoch_num = epoch
                for i in range(epoch_num):
                    myANN.feed_forward()
                    myANN.back_propagation()
                    myANN.cross_entropy_loss()
             
                y_pred = myANN.predict(X_test_norm)
                acc    = accuracy(y_pred, y_test.ravel())
                print('Accuracy of the model ', acc, '\t h1 = ', h1, 'h2 = ', h2, 'lr = ', lr, 'epoch = ', epoch )

Accuracy of the model  0.882 	 h1 =  50 h2 =  50 lr =  0.0005 epoch =  50
Accuracy of the model  0.9 	 h1 =  50 h2 =  50 lr =  0.0005 epoch =  100
Accuracy of the model  0.882 	 h1 =  50 h2 =  50 lr =  0.0005 epoch =  200
Accuracy of the model  0.888 	 h1 =  50 h2 =  50 lr =  0.0005 epoch =  300
Accuracy of the model  0.902 	 h1 =  50 h2 =  50 lr =  0.001 epoch =  50
Accuracy of the model  0.892 	 h1 =  50 h2 =  50 lr =  0.001 epoch =  100
Accuracy of the model  0.908 	 h1 =  50 h2 =  50 lr =  0.001 epoch =  200
Accuracy of the model  0.89 	 h1 =  50 h2 =  50 lr =  0.001 epoch =  300
Accuracy of the model  0.788 	 h1 =  50 h2 =  50 lr =  0.005 epoch =  50
Accuracy of the model  0.818 	 h1 =  50 h2 =  50 lr =  0.005 epoch =  100
Accuracy of the model  0.796 	 h1 =  50 h2 =  50 lr =  0.005 epoch =  200
Accuracy of the model  0.798 	 h1 =  50 h2 =  50 lr =  0.005 epoch =  300
Accuracy of the model  0.768 	 h1 =  50 h2 =  50 lr =  0.01 epoch =  50
Accuracy of the model  0.778 	 h1 =  50 h2

The best accuracy is 0.92

number of neurons in the first hidden layer:    100 

number of neurons in the second hidden layer:   200

learning rate:  0.001

epoch size:     100


## Question 2
```
python CNN.py   --kernel_1_n    <# of kernels in the first convolutional layer>
                --kernel_1_n    <# of kernels in the second convolutional layer>
                --fc            <# of neurons in the linear layer>
                --lr            <learning rate>
                --epochs        <epoch size>
```

In [1]:
from CNN import *
# read the data
X_train, y_train = read_dataset('./data/MNIST_X_train.csv', './data/MNIST_y_train.csv')
X_test, y_test = read_dataset('./data/MNIST_X_test.csv', './data/MNIST_y_test.csv')
X_train, X_test = normalize_features(X_train, X_test)

#==================================Pack Data================================
train_data = torch.from_numpy(X_train).float()
test_data = torch.from_numpy(X_test).float()

trainset = TensorDataset(train_data, torch.from_numpy(y_train.ravel()))
testset = TensorDataset(test_data, torch.from_numpy(y_test.ravel()))

# Define data loader
train_loader = DataLoader(dataset=trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=testset, batch_size=X_test.shape[0], shuffle=False)



In [6]:

# tune the hyper parameters
for kernel_1_n in [4,8,16]:
    for kernel_2_n in [4,8,16]:
        for fc in [50,100,200]:
            for lr in [0.0005,0.001,0.005]:
                for epochs in [30,50,100]:
                    model = CNN_model(d_input=X_train.shape[2], d_output=10,
                                kernel_1_n=kernel_1_n,
                                kernel_2_n=kernel_2_n,
                                fc_n=fc)

                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    lr_adjust = optim.lr_scheduler.StepLR(optimizer,
                                step_size = 20, gamma = 0.5, last_epoch = -1)

                    # train
                    for epoch in range(1, epochs + 1):
                        lr_adjust.step()
                        model.train()
                        for batch_idx, (data, target) in enumerate(train_loader):
                            output = model(data)
                            loss = F.nll_loss(output, target)
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        


                    # testing
                    model.eval()
                    correct = 0
                    with torch.no_grad():
                        for data, target in test_loader:
                            output = model(data)
                            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
                            correct += pred.eq(target.view_as(pred)).sum().item()
                    acc = 100. * correct / len(test_loader.dataset)
                    print('Accuracy of the model', acc, '\t kernel_1_n = ', kernel_1_n, 'kernel_2_n = ', kernel_2_n, 'fc = ', fc, 'lr = ', lr, 'epochs = ', epochs)



Accuracy of the model 93.8 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.0005 epochs =  30
Accuracy of the model 94.0 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.0005 epochs =  50
Accuracy of the model 92.2 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.0005 epochs =  100
Accuracy of the model 91.8 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.001 epochs =  30
Accuracy of the model 92.6 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.001 epochs =  50
Accuracy of the model 93.2 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.001 epochs =  100
Accuracy of the model 93.4 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  30
Accuracy of the model 95.2 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  50
Accuracy of the model 94.2 	 kernel_1_n =  4 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  100
Accuracy of the model 92.0 	 kernel_1_n =  4 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  30
Accuracy of the model 94.0 	 kernel_1_n = 

Accuracy of the model 93.0 	 kernel_1_n =  8 kernel_2_n =  4 fc =  50 lr =  0.001 epochs =  50
Accuracy of the model 94.8 	 kernel_1_n =  8 kernel_2_n =  4 fc =  50 lr =  0.001 epochs =  100
Accuracy of the model 94.4 	 kernel_1_n =  8 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  30
Accuracy of the model 95.6 	 kernel_1_n =  8 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  50
Accuracy of the model 96.2 	 kernel_1_n =  8 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  100
Accuracy of the model 93.0 	 kernel_1_n =  8 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  30
Accuracy of the model 94.0 	 kernel_1_n =  8 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  50
Accuracy of the model 92.8 	 kernel_1_n =  8 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  100
Accuracy of the model 94.8 	 kernel_1_n =  8 kernel_2_n =  4 fc =  100 lr =  0.001 epochs =  30
Accuracy of the model 94.2 	 kernel_1_n =  8 kernel_2_n =  4 fc =  100 lr =  0.001 epochs =  50
Accuracy of the model 95.0 	 kernel_1_n

Accuracy of the model 95.0 	 kernel_1_n =  16 kernel_2_n =  4 fc =  50 lr =  0.005 epochs =  100
Accuracy of the model 93.8 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  30
Accuracy of the model 94.6 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  50
Accuracy of the model 94.4 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.0005 epochs =  100
Accuracy of the model 94.6 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.001 epochs =  30
Accuracy of the model 94.6 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.001 epochs =  50
Accuracy of the model 96.2 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.001 epochs =  100
Accuracy of the model 94.6 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.005 epochs =  30
Accuracy of the model 95.2 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.005 epochs =  50
Accuracy of the model 96.0 	 kernel_1_n =  16 kernel_2_n =  4 fc =  100 lr =  0.005 epochs =  100
Accuracy of the model 94

The best accuracy of the model is 97.2

number of kernels in the first convolutional layer =  16

number of kernels in the second convolutional layer =  16

number of neurons in the fully connected layer =  200 

learning rate =  0.005 

epoch size =  30